In [1]:
api_key="enter_api_key"

In [2]:
!pip install langchain_openai
!pip install OpenAi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 1.7 MB/s eta 0:00:00


In [23]:
import json
import os
import requests
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain import hub

HP_URL = "https://hp-api.onrender.com/api/characters/house/"

os.environ["OPENAI_API_KEY"] = api_key

def personajesHPporcasa(casa):
  """
  Fetches Harry Potter characters belonging to a specific Howarts house, Inputs must be in a single string.
  Como argumento recibe el nombre de una casa
  Ese nombre debe ser un string
  La casa debe ser una casa valida en el universo de Harry Potter
  (e.g., 'gryffindor','slytherin','ravenclaw','hufflepuff' )
  """
  api_url = f"{HP_URL}{casa.lower()}"
  response = requests.get(api_url)
  personajes_json = response.json()
  return json.dumps(personajes_json,indent=2)

def getClima(lugar):
  """
  ESTA FUNCION SIMULA UN LLAMADO DE API PARA DATOS CLIMATICOS. DANDO COMO ENTRADA UN LUGAR, DEVOLVEMOS DATOS DEL CLIMA.
  LA ENTRADA 'lugar' ES UN STRING

  """
  print(f"Buscando el clima de: {lugar}")
  datos_clima = {
      'lugar':lugar,
      "temperatura":'24 celcius',
      "humedad":"58%"


  }
  string_respuesta = json.dumps(datos_clima)
  print(f"Respuesta de Tool: {string_respuesta}")
  return string_respuesta


In [24]:
getClima('Acapulco')

Buscando el clima de: Acapulco
Respuesta de Tool: {"lugar": "Acapulco", "temperatura": "24 celcius", "humedad": "58%"}


'{"lugar": "Acapulco", "temperatura": "24 celcius", "humedad": "58%"}'

In [25]:
tool_clima = Tool(
    name="getClima",
    func=getClima,
    description = """
    Util para obtener el clima de un lugar en particular
    El input debe ser un string que representa al lugar buscado
    SOLO SIRVE PARA EL CLIMA. PREGUTAS QUE NO DE TENGAN QUE VER
    CON EL CLIMA DEBERAN SER RESPONDIDAS CON OTRA HERRAMIENTA!
    """
)

tool_harry=Tool(
    name='personajesHPporcasa',
    func=personajesHPporcasa,
    description= """
  se ocupa para dar datos sobre personajes de Harry Potter pertenecientes
  a una casa en particular
  Como argumento recibe el nombre de una casa
  Ese nombre debe ser un string
  La casa debe ser una casa valida en el universo de Harry Potter
  (e.g., 'gryffindor','slytherin','ravenclaw','hufflepuff' )
  """

)

In [26]:
llm = ChatOpenAI(
    #openai_api_key = OPENAI_API_KEY,
    model_name = "gpt-3.5-turbo"
)

In [27]:
prompt = hub.pull("hwchase17/react-chat")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [28]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat', 'lc_hub_commit_hash': '3ecd5f710db438a9cf3773c57d6ac8951eefd2cd9a9b2a0026a65a0893b86a6e'}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative res

In [29]:
prompt + """
EXTRA:
- No usar la herrameinta getClima si la pregunta no tiene que ver con saber el clima de un lugar
- Usa tools para preguntas que tengan que ver con Harry Potter, NO USES CONOCIMIENTO INTERNO
"""

PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussi

In [30]:
agent = create_react_agent(
    llm = llm,
    tools = [tool_clima, tool_harry],
    prompt=prompt
)

In [31]:
agent_executor = AgentExecutor(
    agent = agent,
    tools = [tool_clima, tool_harry],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations = 5
)

In [35]:
query = "Cuales son los nombres alternativos de personajes de slytherin"
result = agent_executor.invoke({"input":query,"chat_history":[]})
result



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: personajesHPporcasa
Action Input: slytherin[
  {
    "id": "af95bd8a-dfae-45bb-bc69-533860d34129",
    "name": "Draco Malfoy",
    "alternate_names": [],
    "species": "human",
    "gender": "male",
    "house": "Slytherin",
    "dateOfBirth": "05-06-1980",
    "yearOfBirth": 1980,
    "wizard": true,
    "ancestry": "pure-blood",
    "eyeColour": "grey",
    "hairColour": "blonde",
    "wand": {
      "wood": "hawthorn",
      "core": "unicorn tail hair",
      "length": 10
    },
    "patronus": "",
    "hogwartsStudent": true,
    "hogwartsStaff": false,
    "actor": "Tom Felton",
    "alternate_actors": [],
    "alive": true,
    "image": "https://ik.imagekit.io/hpapi/draco.jpg"
  },
  {
    "id": "3569d265-bd27-44d8-88e8-82fb0a848374",
    "name": "Severus Snape",
    "alternate_names": [
      "Half-Blood Prince",
      "Snivellus",
      "Sev"
    ],
    "species": "human",
    "gender": "male

{'input': 'Cuales son los nombres alternativos de personajes de slytherin',
 'chat_history': [],
 'output': 'Some of the alternate names of characters from Slytherin are:\n1. Severus Snape: Half-Blood Prince, Snivellus, Sev\n2. Bellatrix Lestrange: Bella\n3. Lord Voldemort: Tom Riddle, Tom Marvolo Riddle, You-Know-Who, He Who Must Not Be Named, the Dark Lord, the Heir of Slytherin'}

In [36]:
result['output']

'Some of the alternate names of characters from Slytherin are:\n1. Severus Snape: Half-Blood Prince, Snivellus, Sev\n2. Bellatrix Lestrange: Bella\n3. Lord Voldemort: Tom Riddle, Tom Marvolo Riddle, You-Know-Who, He Who Must Not Be Named, the Dark Lord, the Heir of Slytherin'